Prédire la qualité de l’air (AQI Value) à partir des moyennes (Avg) des polluants.

In [7]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


1. Préparation des Données: 


- Charger et parser le fichier XML
- Créer un DataFrame

In [8]:
#Importer les librairies nécessaires
import pandas as pd
import xml.etree.ElementTree as ET

In [9]:
def xml_to_dataframe(xml_file):

    # Charger et parser le fichier
    tree = ET.parse(xml_file)  # adapte le nom du fichier
    root = tree.getroot()

    data = []
    # Explorer tous les éléments station

    for country in root.findall('.//Country'):
        for state in country.findall('.//State'):
            state_name = state.attrib.get('id', None)  
            for city in state.findall('.//City'):
                city_name = city.attrib.get('id', None)
                for station in city.findall('.//Station'):
                    station_name = station.attrib.get('id', None)
                    latitude = station.attrib.get('latitude', None)
                    longitude = station.attrib.get('longitude', None)
                    
                    station_data = {
                        'Country': country.attrib.get('id', None),  
                        'State': state_name,                       
                        'City': city_name,
                        'Station': station_name,
                        'Latitude': latitude,
                        'Longitude': longitude
                    }

                    # Polluants
                    for pollutant in station.findall('Pollutant_Index'):
                        pol_name = pollutant.attrib['id']
                        avg = pollutant.attrib['Avg']
                        station_data[f"{pol_name}_Avg"] = float(avg) if avg != "NA" else None

                    # AQI
                    aqi = station.find('Air_Quality_Index')
                    if aqi is not None:
                        station_data['AQI'] = int(aqi.attrib['Value'])
                        station_data['Predominant_Pollutant'] = aqi.attrib['Predominant_Parameter']
                        data.append(station_data)


    # Créer le DataFrame
    df = pd.DataFrame(data)
    return df

In [10]:
df = xml_to_dataframe('data_aqi_cpcb.xml')

#afficher le DataFrame
print(df.head())

  Country           State               City  \
0   India  Andhra_Pradesh          Anantapur   
1   India  Andhra_Pradesh           Chittoor   
2   India  Andhra_Pradesh             Kadapa   
3   India  Andhra_Pradesh  Rajamahendravaram   
4   India  Andhra_Pradesh           Tirupati   

                                          Station    Latitude   Longitude  \
0                    Gulzarpet, Anantapur - APPCB   14.675886   77.593027   
1             Gangineni Cheruvu, Chittoor - APPCB   13.204880   79.097889   
2                 Yerramukkapalli, Kadapa - APPCB   14.465052   78.824187   
3  Anand Kala Kshetram, Rajamahendravaram - APPCB  16.9872867  81.7363176   
4                      Tirumala, Tirupati - APPCB       13.67       79.35   

   PM2.5_Avg  PM10_Avg  NO2_Avg  NH3_Avg  SO2_Avg  CO_Avg  OZONE_Avg  AQI  \
0       24.0      24.0     23.0      3.0      7.0    29.0       25.0   29   
1       19.0      22.0     14.0      2.0      9.0     4.0       35.0   35   
2       37.0     

In [11]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country                454 non-null    object 
 1   State                  454 non-null    object 
 2   City                   454 non-null    object 
 3   Station                454 non-null    object 
 4   Latitude               454 non-null    object 
 5   Longitude              454 non-null    object 
 6   PM2.5_Avg              448 non-null    float64
 7   PM10_Avg               442 non-null    float64
 8   NO2_Avg                436 non-null    float64
 9   NH3_Avg                388 non-null    float64
 10  SO2_Avg                425 non-null    float64
 11  CO_Avg                 437 non-null    float64
 12  OZONE_Avg              426 non-null    float64
 13  AQI                    454 non-null    int64  
 14  Predominant_Pollutant  454 non-null    object 
dtypes: flo

,PM2.5_Avg,PM10_Avg,NO2_Avg,NH3_Avg,SO2_Avg,CO_Avg,OZONE_Avg,AQI
count,448.000000,442.000000,436.000000,388.000000,425.000000,437.000000,426.000000,454.000000
mean,151.852679,132.361991,33.993119,6.974227,16.124706,47.048055,28.504695,161.555066
std,105.825806,76.433661,30.773215,7.368440,15.105009,27.342962,28.889912,99.428323
min,10.000000,16.000000,2.000000,1.000000,1.000000,4.000000,1.000000,15.000000
25%,63.000000,78.000000,14.000000,3.000000,7.000000,28.000000,12.000000,81.000000
50%,122.500000,117.000000,26.000000,5.000000,12.000000,41.000000,21.000000,137.000000
75%,232.500000,167.750000,44.000000,9.000000,19.000000,59.000000,35.000000,234.750000
max,416.000000,419.000000,245.000000,85.000000,126.000000,170.000000,220.000000,419.000000


In [12]:
df.shape

(454, 15)

2.	Extraire les données pertinentes (features + label)

In [13]:
#Les features (X) sont : PM2.5_Avg, PM10_Avg, NO2_Avg, NH3_Avg, SO2_Avg, CO_Avg, OZONE_Avg
# Le label (y) est : AQI (la valeur à prédire)
features_cols = ['PM2.5_Avg', 'PM10_Avg', 'NO2_Avg', 'NH3_Avg', 'SO2_Avg', 'CO_Avg', 'OZONE_Avg']
label_col = 'AQI'


3.	Nettoyer et structurer les données:

    - Convertir les données en valeurs numériques;
    - Supprimer les lignes incomplètes


In [14]:
# Convertir en float tous les polluants
for col in features_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # convertit 'NA' en NaN

# Garder que les lignes complètes 
df_clean = df.dropna(subset=features_cols + [label_col]) 


4.	Séparer les données en features (X) et cibles (y)

In [15]:
X = df_clean[features_cols]
y = df_clean[label_col]


5. Diviser les données en ensembles d'entraînement et de test


In [16]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [17]:
# Importer les librairies nécessaires
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(278, 7)
(70, 7)
(278,)
(70,)


6. Entraîner le modèle de régression

a.  Le modele RandomForestRegressor

In [35]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [45]:
#Évaluer le modèle de régression
#Prédire avec le modèle sur l'ensemble d'entraînement: prédire les valeurs de AQI pour l’ensemble d'entraînement
y_train_pred = model.predict(X_train)

#Évaluer sur l’ensemble d'entraînement:
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)

print('La performance du modèle RandomForestRegressor sur la base dapprentissage:')
print(f"RMSE (train) : {rmse_train}")
print(f"R² (train)   : {r2_train}")

#Prédire avec le modèle sur l’ensemble de test : prédire les valeurs de AQI pour l’ensemble de test 
y_test_pred = model.predict(X_test)

#Évaluer sur l’ensemble de test:
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)

print('La performance du modèle RandomForestRegressor sur la base de test:')
print(f"RMSE (test) : {rmse_test}")
print(f"R² (test)   : {r2_test}")

La performance du modèle RandomForestRegressor sur la base dapprentissage:
RMSE (train) : 2.995649543429665
R² (train)   : 0.999086803869418
La performance du modèle RandomForestRegressor sur la base de test:
RMSE (test) : 14.643969504787385
R² (test)   : 0.9759351289254019


b. Le modele de regression linéaire

In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Créer le modèle
linear_model = LinearRegression()

# Entraîner sur l'ensemble d'entraînement
linear_model.fit(X_train, y_train)

# Prédire sur l'ensemble d'entraînement
y_train_pred_lr = linear_model.predict(X_train)

# Évaluer sur l'ensemble d'entraînement
rmse_train_lr = np.sqrt(mean_squared_error(y_train, y_train_pred_lr))
r2_train_lr = r2_score(y_train, y_train_pred_lr)

# Prédire sur l'ensemble de test
y_test_pred_lr = linear_model.predict(X_test)

# Évaluer sur l'ensemble de test
rmse_test_lr = np.sqrt(mean_squared_error(y_test, y_test_pred_lr))
r2_test_lr = r2_score(y_test, y_test_pred_lr)

# Afficher les résultats
print('La performance du modèle LinearRegression sur la base dapprentissage:')
print(f"RMSE (train): {rmse_train_lr}")
print(f"R² (train)  : {r2_train_lr}")

print('La performance du modèle LinearRegression sur la base de test:')
print(f"RMSE (test) : {rmse_test_lr}")
print(f"R² (test)   : {r2_test_lr}")


La performance du modèle LinearRegression sur la base dapprentissage:
RMSE (train): 12.503378480900034
R² (train)  : 0.9840912224670736
La performance du modèle LinearRegression sur la base de test:
RMSE (test) : 13.571782405943889
R² (test)   : 0.9793300377336192
